In [1]:
import papermill as pm

In [2]:
import psutil

# Get memory information
memoryInfo = psutil.virtual_memory()
total_memory = memoryInfo.total / (1024 ** 3)
available_memory = memoryInfo.available / (1024 ** 3)
used_memory = memoryInfo.used / (1024 ** 3)

print(f"Total Memory: {total_memory:.2f} GB")
print(f"Available Memory: {available_memory:.2f} GB")
print(f"Used Memory: {used_memory:.2f} GB")

Total Memory: 125.58 GB
Available Memory: 121.79 GB
Used Memory: 2.58 GB


In [3]:
def delete_ct_data(sample_dir):

    from pathlib import Path
    import shutil
    
    # Define the paths for the directory and the file
    directory_path = Path(sample_dir + 'JPEG/')
    file_path = Path(sample_dir + 'microCT_volume/microCT_volume.nrrd')
    
    # Check if the directory exists and delete it
    if directory_path.exists() and directory_path.is_dir():
        shutil.rmtree(directory_path)  # Remove the directory and all its contents
        print(str(directory_path) + " deleted!")
            
    # Check if the file exists and delete it
    if file_path.exists() and file_path.is_file():
        file_path.unlink()  # Remove the file
        print(str(file_path) + " deleted!")

In [19]:
def run_processing_workflow(sample_dir):
    
    # Run `00_tif2nrrd_previewQuality.ipynb`
    paper_dict = dict(image_directory_name = sample_dir + 'JPEG',
                      output_dir_path = sample_dir + 'microCT_volume'
                     )
    
    input_nb = '00_tif2nrrd_previewQuality.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));
    
    # Run `00_tif2nrrd_fullQuality.ipynb`
    paper_dict = dict(image_directory_name = sample_dir + 'JPEG',
                      output_dir_path = sample_dir + 'microCT_volume'
                     )
    
    input_nb = '00_tif2nrrd_fullQuality.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));
    
    # Run `02_find_threshold_segment.ipynb`
    paper_dict = dict(volume_file = sample_dir + 'microCT_volume/microCT_volume.nrrd',
                      output_dir_path = sample_dir + 'segmented_volumes/',
                     )
    
    input_nb = '02_find_threshold_segment.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));
    
    # Run `03_segment_microCT.ipynb`
    paper_dict = dict(volume_file = sample_dir + 'microCT_volume/microCT_volume.nrrd',
                      output_dir_path = sample_dir + 'segmented_volumes/',
                     )
    
    input_nb = '03_segment_microCT.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict)); 
    
    # Run `04_scaffoldSegment_to_stlFile.ipynb`
    paper_dict = dict(volume_file = sample_dir + 'microCT_volume/microCT_volume_preview.nrrd',
                      output_dir_path = sample_dir + 'segmented_volumes/'
                      )
    
    input_nb = '04_segments_2stlFiles.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict), timeout=1200);  # Set timeout to 1200 seconds (20 minutes)

    # Run `05_define_scaffold_cylinders.ipynb`
    paper_dict = dict(segmented_dir =  sample_dir +'segmented_volumes/',
                      volume_file = sample_dir + 'microCT_volume/microCT_volume.nrrd',
                      scaffold_filename = sample_dir + 'segmented_volumes/Segmentation_Scaffold.stl',
                      output_cylinders_dir = sample_dir + 'segmented_volumes/cylinders/'
                      )
    
    input_nb = '05_define_scaffold_cylinders.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));
    

    # Run `06_bone_ingrowth_scaffold-cylinder.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      cylinders_dir = sample_dir + 'segmented_volumes/cylinders/',
                      bone_cylinder_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd',
                      model_name = 'Scaffold-Cylinder'
                      )
    
    
    
    input_nb = '06_bone_ingrowth_scaffold-cylinder.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));    

    
    # Run `07_bone_ingrowth_outer-cylinder.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      cylinders_dir = sample_dir + 'segmented_volumes/cylinders/',
                      bone_cylinder_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd',
                      model_name = 'Cylinder-Outer'
                      )
    
    
    
    input_nb = '06_bone_ingrowth_scaffold-cylinder.ipynb'
    output_nb =  sample_dir + '07_bone_ingrowth_outer-cylinder.ipynb'
    
    print(output_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));    
    
    # Run `08_bone_ingrowth_middle-cylinder.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      cylinders_dir = sample_dir + 'segmented_volumes/cylinders/',
                      bone_cylinder_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd',
                      model_name = 'Cylinder-Middle'
                      )
    
    
    
    input_nb = '06_bone_ingrowth_scaffold-cylinder.ipynb'
    output_nb =  sample_dir + '08_bone_ingrowth_middle-cylinder.ipynb' 
    
    print(output_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));   
    
    # Run `09_bone_ingrowth_inner-cylinder.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      cylinders_dir = sample_dir + 'segmented_volumes/cylinders/',
                      bone_cylinder_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd',
                      model_name = 'Cylinder-Inner'
                      )
    
    
    
    input_nb = '06_bone_ingrowth_scaffold-cylinder.ipynb'
    output_nb =  sample_dir + '09_bone_ingrowth_inner-cylinder.ipynb' 
    
    print(output_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));
    
    # Run `10_bone_ingrowth_volumes_total.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      volume_file = sample_dir + 'microCT_volume/microCT_volume.nrrd',
                      bone_cylinders_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd'
                      )
    
    input_nb = '10_bone_ingrowth_volumes_total.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));

    # Run `11_bone_ingrowth_volumes_ingrowth.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      volume_file = sample_dir + 'microCT_volume/microCT_volume.nrrd',
                      bone_cylinders_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd'
                      )
    
    input_nb = '11_bone_ingrowth_volumes_ingrowth.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));


    # Run `12_bone_ingrowth_volumes_outer.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      volume_file = sample_dir + 'microCT_volume/microCT_volume.nrrd',
                      bone_cylinders_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd'
                      )
    
    input_nb = '12_bone_ingrowth_volumes_outer.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));
    
    # Run `13_bone_ingrowth_volumes_middle.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      volume_file = sample_dir + 'microCT_volume/microCT_volume.nrrd',
                      bone_cylinders_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd'
                      )
    
    input_nb = '13_bone_ingrowth_volumes_middle.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));
    
    # Run `14_bone_ingrowth_volumes_inner.ipynb`
    paper_dict = dict(sample_name = sample_dir.split('/')[-2], # 'GB3_8W',
                      segmented_dir = sample_dir + 'segmented_volumes/',
                      volume_file = sample_dir + 'microCT_volume/microCT_volume.nrrd',
                      bone_cylinders_dir = sample_dir + 'segmented_volumes/bone_cylinders/',
                      file_segmentation = sample_dir + 'segmented_volumes/Bone.seg.nrrd'
                      )
    
    input_nb = '14_bone_ingrowth_volumes_inner.ipynb'
    output_nb =  sample_dir + input_nb
    
    print(input_nb)
    pm.execute_notebook(input_nb, output_nb, parameters=dict(paper_dict));

In [5]:
sample_dir = '4_weeks/GB4_4W/'

run_processing_workflow(sample_dir)

delete_ct_data(sample_dir)

4_weeks/GB4_4W/07_bone_ingrowth_outer-cylinder.ipynb


Executing: 100%|##########| 25/25 [01:46<00:00,  4.24s/cell]


4_weeks/GB4_4W/08_bone_ingrowth_middle-cylinder.ipynb


Executing: 100%|##########| 25/25 [01:45<00:00,  4.21s/cell]


4_weeks/GB4_4W/09_bone_ingrowth_inner-cylinder.ipynb


...

13_bone_ingrowth_volumes_middle.ipynb


Executing: 100%|##########| 42/42 [02:26<00:00,  3.49s/cell]


14_bone_ingrowth_volumes_inner.ipynb


Executing: 100%|##########| 42/42 [02:22<00:00,  3.40s/cell]


4_weeks/GB4_4W/JPEG deleted!
4_weeks/GB4_4W/microCT_volume/microCT_volume.nrrd deleted!
